# Tribble Saldo

calculate workings hours saldo.

In [59]:
time_span="01.01.2023-31.12.2023"
country="germany"
territory="B"

In [61]:
import os
from datetime import datetime, timedelta
import pytz
from dateutil.easter import easter

reference_functions = {
    "easter": easter
}

day_span_list = time_span.split('-')

day_span_begin_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(day_span_list[0], "%d.%m.%Y"))
day_span_end_local = pytz.timezone('Europe/Berlin').localize(datetime.strptime(day_span_list[1], "%d.%m.%Y") + timedelta(days=1))

year_begin_local = day_span_begin_local.year
year_end_local = day_span_end_local.year

day_span_begin_utc = day_span_begin_local.astimezone(pytz.utc)
day_span_end_utc = day_span_end_local.astimezone(pytz.utc)


APPLICATION_PATH = os.getenv('PURRING_TRIBBLE_HOME_APPLICATION')

## get holidays

In [62]:
import json
from datetime import datetime, timedelta
import pytz

def check_holiday_in_current_country(country, territory, holiday_dict):
    for country_dict in holiday_dict['countries']:
        if country == country_dict['name'] and territory in country_dict['territories']:
            return True
    return False

with open(f"{APPLICATION_PATH}/public_holidays.json", 'r') as f:
  public_holidays_dict = json.load(f)

holidays = []

for current_year in range(year_begin_local, year_end_local):
    for holiday_dict in public_holidays_dict:
        if holiday_dict['type'] == 'fix' and check_holiday_in_current_country(country, territory, holiday_dict):
            day = datetime.strptime(f"{holiday_dict['day']}.{current_year}", "%d.%m.%Y").date()
        elif holiday_dict['type'] == 'calc' and check_holiday_in_current_country(country, territory, holiday_dict):
            day = reference_functions[holiday_dict['formula']['reference']](current_year) + timedelta(days=holiday_dict['formula']['difference'])
        holidays.append(day)

## analyze workdays

In [93]:
import os
from datetime import datetime, timedelta
import sqlite3
import pytz
import json

tribble_db_fqn = f"{os.getenv('PURRING_TRIBBLE_DATABASE')}"

con = sqlite3.connect(tribble_db_fqn)

cur = con.cursor()

def is_workday(current_day):
    if current_day.date() in holidays:
        return False
    return True

def get_target_working_hours(current_day):
    if not is_workday(current_day):
        return timedelta(hours=0)
    current_day_utc = current_day.astimezone(pytz.utc)
    sql_statement = f"SELECT hours_distribution FROM worktime_hours w WHERE DATETIME(timeslot_begin) <= DATETIME('{current_day_utc.strftime('%Y-%m-%d %H:%M:%S')}') AND DATETIME(timeslot_end) > DATETIME('{current_day_utc.strftime('%Y-%m-%d %H:%M:%S')}');"
    res = cur.execute(sql_statement)
    results_list = res.fetchall()
    working_hours_dict = json.loads(results_list[0][0])
    return timedelta(hours=working_hours_dict[str(datetime.weekday(current_day))])

def get_hours_multiplicator(current_day):
    current_day_utc = current_day.astimezone(pytz.utc)
    sql_statement = f"SELECT worktime_multiplicator FROM worktime_special w WHERE DATETIME(timeslot_begin) <= DATETIME('{current_day_utc.strftime('%Y-%m-%d %H:%M:%S')}') AND DATETIME(timeslot_end) > DATETIME('{current_day_utc.strftime('%Y-%m-%d %H:%M:%S')}');"
    res = cur.execute(sql_statement)
    results_list = res.fetchall()
    if len(results_list) > 0:
        return results_list[0][0]
    return 1

def get_worked_hours(current_day):
    current_day_utc = current_day.astimezone(pytz.utc)
    sql_statement = f"SELECT timeslot_begin, timeslot_end FROM worktime w WHERE DATETIME(timeslot_begin) >= DATETIME('{current_day_utc.strftime('%Y-%m-%d %H:%M:%S')}') AND DATETIME(timeslot_begin) < DATETIME('{(current_day_utc + timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')}') AND symbol == 'common' AND timeslot_finish == TRUE;"
    res = cur.execute(sql_statement)
    results_list = res.fetchall()
    if len(results_list) > 0:
        difference = timedelta(hours=0)
        for timeslot in results_list:
            difference = difference + (datetime.strptime(timeslot[1], "%Y-%m-%d %H:%M:%S") - datetime.strptime(timeslot[0], "%Y-%m-%d %H:%M:%S"))
        return difference
    return timedelta(hours=0)

workday_list = []
current_day = day_span_begin_local
while current_day <= day_span_end_local:
    current_workday_dict = {
        "datetime": current_day,
        "target_hours": get_target_working_hours(current_day),
        "worked_hours": get_worked_hours(current_day),
        "multiplicator": get_hours_multiplicator(current_day)
    }
    current_workday_dict["needed_hours"] = current_workday_dict["target_hours"] * current_workday_dict["multiplicator"]
    if len(workday_list) == 0:
        saldo_needed_yesterday = timedelta(hours=0)
        saldo_worked_yesterday = timedelta(hours=0)
    else:
        saldo_needed_yesterday = workday_list[len(workday_list)-1]["saldo_needed"]
        saldo_worked_yesterday = workday_list[len(workday_list)-1]["saldo_worked"]
    current_workday_dict["saldo_needed"] = saldo_needed_yesterday + current_workday_dict["needed_hours"]
    current_workday_dict["saldo_worked"] = saldo_worked_yesterday + current_workday_dict["worked_hours"]
    workday_list.append(current_workday_dict)
    current_day = current_day + timedelta(days=1)
#print(len(workday_list))
#print(workday_list)

con.close()

In [94]:
import csv

with open('result.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
    w = csv.DictWriter(f, workday_list[0].keys())
    w.writeheader()
    for current_workday_dict in workday_list:
        w = csv.DictWriter(f, current_workday_dict)
        w.writerow(current_workday_dict)